## Transfer Learning CNN Model

In [27]:
import numpy as np
import pandas as pd
from tensorflow import keras

from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, GlobalAveragePooling2D, Normalization
from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam

from keras.applications import MobileNetV2, InceptionV3, DenseNet121, VGG19
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.losses import sparse_categorical_crossentropy

from keras.callbacks import EarlyStopping

In [1]:
# Constants used in the CNN model

data_dir = 'processed'

IMG_HEIGHT, IMG_WIDTH = 512, 512
TARGET_HEIGHT, TARGET_WIDTH = 224, 224

BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

NUM_CLASSES = 13
EPOCHS = 15

In [4]:
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Art Styles (Class Names): {class_names}")

Found 42385 files belonging to 13 classes.
Using 33908 files for training.
Found 42385 files belonging to 13 classes.
Using 8477 files for validation.
Art Styles (Class Names): ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


In [ ]:
base_model = MobileNetV2(weights='imagenet', 
                      include_top=False, 
                      input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                      pooling=None)

base_model.trainable = True
total_layers = len(base_model.layers)

for layer in base_model.layers[:-30]:
    layer.trainable = False

for layer in base_model.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

model = Sequential([
    InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    Normalization(
        mean=[-1.]*3,
        variance=[1.]*3,
    ),
    base_model
    ])

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],     
    optimizer=Adam(1e-3)
    )
    

/var/folders/dq/9_kf7lnd13lg7428445qjpmh0000gn/T/ipykernel_44343/3640509548.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet',
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [31]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 16, 16, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 13)             │           845 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,431,053 (9.27 MB)

 Trainable params: 1,683,789 (6.42 MB)

 Non-trainable params: 747,264 (2.85 MB)

In [32]:
earlystop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [33]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[earlystop]
)


Epoch 1/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2552s 2s/step - accuracy: 0.1507 - loss: 2.4326 - val_accuracy: 0.1574 - val_loss: 2.4099
Epoch 2/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2776s 3s/step - accuracy: 0.1606 - loss: 2.4043 - val_accuracy: 0.1574 - val_loss: 2.4036
Epoch 3/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2916s 3s/step - accuracy: 0.1602 - loss: 2.4040 - val_accuracy: 0.1574 - val_loss: 2.4038
Epoch 4/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 7081s 7s/step - accuracy: 0.1611 - loss: 2.4035 - val_accuracy: 0.1574 - val_loss: 2.4037
Epoch 5/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 3192s 3s/step - accuracy: 0.1604 - loss: 2.4028 - val_accuracy: 0.1574 - val_loss: 2.4038
Epoch 6/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 3657s 3s/step - accuracy: 0.1610 - loss: 2.4034 - val_accuracy: 0.1574 - val_loss: 2.4036
Epoch 7/15
  34/1060 ━━━━━━━━━━━━━━━━━━━━ 33:55 2s/step - accuracy: 0.1566 - loss: 2.4155

KeyboardInterrupt: 